In [ ]:
# pip install -U binclass-tools

In [9]:
import os
import sys
import inspect

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [7]:
titanic_df = pd.read_csv('/Users/muhammadraza/Documents/GitHub/BIPM/Data Science/Dataset_Titanic.csv')

In [21]:
titanic_df.dropna

<bound method DataFrame.dropna of      PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                 

In [19]:
X = titanic_df[['Pclass', 'Age', 'SibSp', 'Parch']]
y = titanic_df["Survived"]

In [20]:
hospital_df = titanic_df.dropna()

# apply train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=1)

# instantiate plain decision tree classifier
clf = RandomForestClassifier()

# train plain decision tree classifier
clf.fit(X_train, y_train)

# predict for all appointment ids
y_pred = clf.predict(X_test)

ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
features = ['Pclass', 'Age', 'SibSp', 'Parch']
X = titanic_df[features]
y = titanic_df['Survived']

In [ ]:
# Conduct a train-test split with a 30% test ratio and a random state of 42
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)

In [ ]:
# Instantiate a Simple Imputer with a Median Strategy
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy="median")

In [ ]:
# Instantiate a One Hot Encoder that ignores unknown categories
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [ ]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(
    [('ohe', ohe, ['Sex']), 
    ('imputer', imp, ['Age'])],              
    remainder='passthrough'
)

ct.fit_transform(X_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Train a RF classifier
cls = RandomForestClassifier(max_depth=6, oob_score=True, random_state=123)
cls.fit(X_train, y_train)

In [ ]:
# Get prediction probabilities for the train set
train_predicted_proba = cls.predict_proba(X_train)[:,1]

# Get prediction probabilities for the test set
test_predicted_proba = cls.predict_proba(X_test)[:,1]

## Import bctools package

In [ ]:
import bctools as bc

### Plot Roc and PR plot, with isoFbeta curves, for the test set

In [ ]:
ROC_plot, area_under_ROC = bc.curve_ROC_plot(true_y = y_test,
                                             predicted_proba = test_predicted_proba)

In [ ]:
ROC_plot
# or
# ROC_plot.show()

In [ ]:
area_under_ROC

In [ ]:
PR_plot, area_under_PR = bc.curve_PR_plot(true_y = y_test,
                                          predicted_proba = test_predicted_proba,
                                          beta = 1)

In [ ]:
PR_plot

In [ ]:
area_under_PR

### Interactive probabilities violin plot for the test set

In [ ]:
threshold_step = 0.05

violin_plot = bc.predicted_proba_violin_plot(true_y = y_test,
                                             predicted_proba = test_predicted_proba,
                                             threshold_step = threshold_step,
                                             #marker_size =3
                                            )

In [ ]:
violin_plot

### Interactive kernel density estimation curve (or normal distribution curve) plot for the test set

Both plots below

In [ ]:
#curve type parameter can be either 'kde' (default) or 'normal'
threshold_step = 0.05

curve_type = 'kde' #default
density_curve_kde = bc.predicted_proba_density_curve_plot(true_y = y_test,
                                                          predicted_proba = test_predicted_proba,
                                                          threshold_step = threshold_step,
                                                          curve_type = curve_type,
                                                          title = 'Interactive Probabilities Distribution Plot (kde)')

curve_type = 'normal'
density_curve_nor = bc.predicted_proba_density_curve_plot(true_y = y_test,
                                                          predicted_proba = test_predicted_proba,
                                                          threshold_step = threshold_step,
                                                          curve_type = 'normal',
                                                          title = 'Interactive Probabilities Distribution Plot (normal)')

In [ ]:
density_curve_kde.show()
density_curve_nor.show()

### Confusion matrix and metrics analysis for train and test set

In [ ]:
# set params for the train dataset
threshold_step = 0.05
amounts = np.abs(X_train[:, 13])
currency = '$'

In [ ]:
# The function get_cost_dict can be used to define the dictionary of costs.
# It takes as input, for each class, a float or a list of floats.
# Lists must have coherent lenghts

train_cost_dict = bc.get_cost_dict(TN = 0, FP = 10, FN = np.abs(X_train[:, 12]), TP = 0)
train_cost_dict

In [ ]:
# plot confusion matrix and get variable metrics dataframe, invariant metric dataframe and optimal thresholds dataframe.

# cost_dict and amounts, if not given, are set to None and won't be visualized.

cf_fig, var_metrics_df, invar_metrics_df, opt_thresh_df = bc.confusion_matrix_plot(
    true_y = y_train,
    predicted_proba = train_predicted_proba,
    threshold_step = threshold_step,
    amounts = amounts,
    cost_dict = train_cost_dict,
    currency = currency,
    title = 'Interactive Confusion Matrix for the Training Set')

In [ ]:
cf_fig

In [ ]:
# the three dataframes returned
display(var_metrics_df, invar_metrics_df, opt_thresh_df)

In [ ]:
# You can also analyze the test dataset.

threshold_step = 0.05
amounts = np.abs(X_test[:, 13])
currency = '$'

test_cost_dict = bc.get_cost_dict(TN = 0, FP = 10, FN = np.abs(X_test[:, 12]), TP = 0)

In [ ]:
cf_fig, var_metrics_df, invar_metrics_df, opt_thresh_df = bc.confusion_matrix_plot(
    true_y = y_test,
    predicted_proba = test_predicted_proba,
    threshold_step = threshold_step,
    amounts = amounts,
    cost_dict = test_cost_dict,
    currency = currency,
    title = 'Interactive Confusion Matrix for the Testing Set')

In [ ]:
cf_fig

In [ ]:
# the invariant metric dataframe can be obtained directly with
# the function get_invariant_metrics_df from the utilities module

bc.utilities.get_invariant_metrics_df(true_y = y_test,
                                      predicted_proba = test_predicted_proba)

In [ ]:
# for a specific threshold,
# the confusion matrix and a dataframe containing the list of metrics visualized in the first table of
# the interactive confusion matrix plot, can be obtained directly with
# the function get_confusion_matrix_and_metrics_df from the utilities module

conf_matrix, metrics_fixed_thresh_df = bc.utilities.get_confusion_matrix_and_metrics_df(
    true_y = y_test,
    predicted_proba = test_predicted_proba,
    threshold = 0.3 # default = 0.5
)

display(conf_matrix, metrics_fixed_thresh_df)


In [ ]:
# the optimal thresholds dataframe can be obtained directly with
# the function get_subset_optimal_thresholds_df from the thresholds module

# this function requires a list of thresholds instead of the step, for example:
threshold_values = np.round(np.arange(0.05, 1, 0.05), 4) # will generate an array of values from 0 to 1 with step 0.05 (rounded for representation reasons)

# to obtain the threshold that minimizes the cost for this train set, we need a train_cost_dict
train_cost_dict = bc.get_cost_dict(TN = 0, FP = 10,
                                   FN = np.abs(X_train[:, 12]), TP = 0)

bc.thresholds.get_subset_optimal_thresholds_df(threshold_values = threshold_values,
                                                 true_y = y_train,
                                                 predicted_proba = train_predicted_proba,
                                                 cost_dict = train_cost_dict)

In [ ]:
# The previously obtained thresholds maximize the related metric - and minimize the cost - for the given train set

# With the GHOST method we can obtain thresholds that generally optimize given metrics for imbalanced sets of data
# The funciotn get_ghost_optimal_thresholds_df from the thresholds module returns a dataframe with the optimal thresholds
# obtained with GHOST method

# WARNING: could take a while

bc.thresholds.get_ghost_optimal_thresholds_df(optimize_threshold = 'all',
                                                threshold_values = threshold_values,
                                                true_y = y_train,
                                                predicted_proba = train_predicted_proba,
                                                cost_dict = train_cost_dict,
                                                N_subsets = 70, subsets_size = 0.2, with_replacement = False, # default
                                                random_state = 120)

In [ ]:
# to directly optimize a threshold for one specific metric in {'MCC', 'Kappa', 'Fscore'},
# the function get_ghost_optimal_threshold from the thresholds module can be used

# if ThOpt_metrics = Fscore, 3 values will be returned (optimal threshold for beta = 1, for beta = 2 and for beta = 0.5)

bc.thresholds.get_ghost_optimal_threshold(y_train,
                                        train_predicted_proba,
                                        threshold_values,
                                        ThOpt_metrics = 'MCC', # default = 'Kappa'
                                        N_subsets = 70, subsets_size = 0.2, with_replacement = False, # defaults
                                        random_seed = 120)

In [ ]:
# to directly optimize a threshold for minimal cost,
# the function get_ghost_optimal_cost from the thresholds module can be used (cost_dict must be given)

bc.thresholds.get_ghost_optimal_cost(y_train,
                                     train_predicted_proba,
                                     threshold_values,
                                     cost_dict = train_cost_dict,
                                     N_subsets = 70, subsets_size = 0.2, with_replacement = False, # defaults
                                     random_seed = 120)

In [ ]:
# plot "Interactive confusion line chart" and get amount/cost per threshold dataframe and total_amount.

# at least one of cost_dict or amounts must be given
# either cost_dict or amounts, if not given, is set to None and won't be visualized
# when amounts is not given, the total_amount returned will be None

cl_fig, amount_cost_df, total_amount = bc.confusion_linechart_plot(
    true_y = y_test,
    predicted_proba = test_predicted_proba,
    threshold_step =  threshold_step,
    amounts = amounts,
    cost_dict = test_cost_dict,
    currency = currency);

In [ ]:
cl_fig

In [ ]:
# total_amount and dataframe returned
print(f'total amount: {currency}{total_amount}')
amount_cost_df

In [ ]:
# the amount/cost per threshold dataframe can be obtained directly with
# the function get_amounts_cost_df in the utilities module

# this function requires a list of thresholds, instead of the step, for example:
threshold_values = np.arange(0, 1, 0.05) # will generate an array of values from 0 to 1 with step 0.05

# example without amounts
bc.utilities.get_amount_cost_df(
    true_y = y_test,
    predicted_proba = test_predicted_proba,
    threshold_values = threshold_values,
    #amounts = amounts,
    cost_dict = test_cost_dict)

### Custom Interactive Amount/Cost line chart

In [ ]:
# plot "Amount/Cost line chart" and get a dataframe containing amount and cost per threshold for selected
# "confusion classes" (TN, FP, FN, TP) and their total

# at least one of cost_dict or amounts must be given
# either cost_dict or amounts, if not given, is set to None and won't be visualized
# amount_classes, if not given, is set to 'all' when amounts is given, to None otherwise
# cost_classes, if not given, is set to 'all' when cost_dict is given, to None otherwise

# for example, if we want to plot the sum of the amounts of the True Positive and False Positive data
# and the sum of the costs of all the data:

amount_classes = ['TP', 'FP']
cost_classes = 'all'

ac_fig, total_cost_amount_df = bc.total_amount_cost_plot(
    true_y = y_test,
    predicted_proba = test_predicted_proba,
    threshold_step = threshold_step,
    amounts = amounts,
    cost_dict = test_cost_dict,
    amount_classes = amount_classes,
    cost_classes = cost_classes,
    currency = currency)

ac_fig

In [ ]:
# dataframe returned by the function
total_cost_amount_df

### Additional useful function

In [ ]:
# the function get_confusion_class_df takes in input a "confusion class" {'TN', 'FP', 'FN', 'TP'},
# a feature dataset (X), the true labels (y), the predicted probabilites and a threshold
# and returns the portion of the feature dataset corresponding to the given class

# for example, if we want the True Positive data points with a 0.7 threshold:
confusion_category = 'TP'

bc.get_confusion_category_observations_df(
    confusion_category = confusion_category,
    X_data = X_test,
    true_y = y_test,
    predicted_proba = test_predicted_proba,
    threshold = 0.7 # default = 0.5
)

### Gain, Lift, Response and Cumulative Response Plot

In [ ]:
cumgain_plot = bc.cumulative_gain_plot(true_y = y_test,
                                       full_predicted_proba = cls.predict_proba(X_test),
                                       pos_label = 1,
                                       )

In [ ]:
cumgain_plot

In [ ]:
lift_curve = bc.lift_curve_plot(true_y = y_test,
                                full_predicted_proba = cls.predict_proba(X_test),
                                pos_label = 1,
                                )

lift_curve

In [ ]:
cumres_plot = bc.cumulative_response_plot(true_y = y_test,
                                          predicted_proba = test_predicted_proba,
                                          )

cumres_plot

In [ ]:
resp_curve = bc.response_curve_plot(true_y = y_test,
                                    predicted_proba = test_predicted_proba,
                                    n_tiles = 10,
                                    )

resp_curve

## Calibration

In [ ]:
calib_curve, ece = bc.calibration_curve_plot(true_y = y_test,
                                             predicted_proba = test_predicted_proba,
                                             n_bins = 10,           #default
                                             strategy = 'uniform',  #default
                                             show_gaps = True,      #default
                                             ece_bins = 'fd'        #default
                                            )

calib_curve

In [ ]:
ece

In [ ]:
#ece can be directly obtained with:
bc.utilities.get_expected_calibration_error(true_y = y_test,
                                            predicted_proba = test_predicted_proba,
                                            bins = 'fd'           #default
                                           )

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

lr = LogisticRegression(C=1.0)
gnb = GaussianNB()

clf_list = [lr, gnb]

for clf in clf_list:
    clf.fit(X_train, y_train)

In [ ]:
line_fig, hist_fig, ece_ls = bc.calibration_plot_from_models(X = X_test,
                                                     true_y = y_test,
                                                     estimators = [cls, lr, gnb],
                                                     estimator_names = ["Random Forest", "Logistic", "Naive Bayes"],
                                                     n_bins = 10,           #default
                                                     strategy = 'uniform',  #default
                                                     ece_bins = 'fd'        #default
                                                    )

In [ ]:
line_fig.show()
hist_fig.show()

In [ ]:
ece_ls